# Not Scaled Data

In [21]:
import pandas as pd
xls = pd.ExcelFile("Фрустрация.xlsx")
data = pd.read_excel(xls, xls.sheet_names)    

In [22]:
keys = [' E', ' E1', ' e_', ' M', ' M1', ' m_', ' I', ' I1', ' i_']

keys1 = [[' E', ' E1', ' e_'],
         [' M', ' M1', ' m_'],
         [' I', ' I1', ' i_']]

keys2 = [[' E', ' M', ' I'],
        [' E1', ' M1', ' I1'],
        [' e_', ' m_', ' i_']]
def get_y(keys, data, init_keys = keys):
    y = []
    for ind, i in enumerate(init_keys):
        for ind1, j in enumerate(keys):
            if i in j:
                y.append(ind1 * np.ones(len(data[i])))
    return np.hstack(y)

In [23]:
data[keys[0]].columns

Index(['Unnamed: 0', 'Число восклицательных знаков',
       'Число вопросительных знаков', 'Число слов', 'Число символов',
       'Число уникальных слов', 'Число клауз', 'Число предложений',
       'Число знаков препинания',
       'Средняя длина предложения (в количестве слов)',
       ...
       'Синтакс. связь: nummod', 'Синтакс. связь: nummod:gov',
       'Синтакс. связь: obj', 'Синтакс. связь: obl', 'Синтакс. связь: orphan',
       'Синтакс. связь: parataxis', 'Синтакс. связь: punct',
       'Синтакс. связь: root', 'Синтакс. связь: xcomp',
       'Синтакс. связь: compound'],
      dtype='object', length=243)

In [24]:
def extract(list_of_texts):
    res = []
    for s in list_of_texts:
        k = 0
        for c in s:
            if c.islower():
                k += 1
        res.append(k/len(s))
    
    return pd.DataFrame({"Доля":res})

In [25]:
import json
with open("p3_texts.json", "r") as f:
    texts = f.read()
    texts = json.loads(texts)
    f.close()
print(texts.keys())
for key in keys:
    if key[-1] == '1':
        key1 = key[:-1] + "'"
    else:
        key1 = key
    data[key] = data[key].join( extract(texts['P3:' + key1]), rsuffix = "___")

dict_keys(['P3: E', 'P3: M', 'P3: m_', 'P3: e_', "P3: I'", "P3: E'", 'P3: i_', "P3: M'", 'P3: инф', 'P3: I', 'P3: n ("надо")'])


In [26]:
import numpy as np
from sklearn.preprocessing import scale
X = np.vstack([data[i].values for i in keys])
X = X[:, 1:]
y1 = get_y(keys1, data)
y2 = get_y(keys2, data)

In [27]:
print("Число слов", X[:, 2].mean())

Число слов 36.49063670411985


In [28]:
print("Число предложений", X[:, 6].mean())

Число предложений 3.4569288389513106


In [29]:
print("Число символов", X[:, 3].mean())

Число символов 188.49812734082397


In [92]:
X = scale(X)
# X_bin = np.where(X <= 0, 0, 1)

## The First Union

In [108]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
model = SelectFromModel(LogisticRegression(C = 0.2, penalty="l1", multi_class = 'ovr', solver = 'liblinear'))
model.fit(X, y1)
X_new = model.transform(X)
print(X.shape)
X_new.shape

(267, 243)


(267, 61)

In [109]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

s = 0
N = 1000
for i in range(N):
    X_, y = shuffle(X_new, y1)
    model = LogisticRegression(C = 0.05, penalty = "l2", solver = "liblinear", class_weight = "balanced", multi_class = "ovr", max_iter = 100)
    s+= cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()
s/N

0.4914439692992847

# Deep Learning Part

In [121]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit(y1)
y = lb.transform(y1)

In [125]:
import torch
from torch import nn
from torch.functional import F
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [157]:
class Model(nn.Module):
    def __init__(self, n_feat):
        super(Model, self).__init__()
        self.model = []
        self.model.append(nn.Linear(in_features=n_feat, out_features=100))
        self.model.append(nn.Dropout())
        self.model.append(nn.ReLU())
        self.model.append(nn.BatchNorm1d(100))
        self.model.append(nn.Linear(in_features=100, out_features=1000))
        self.model.append(nn.Dropout())
        self.model.append(nn.ReLU())
        self.model.append(nn.BatchNorm1d(1000))
        self.model.append(nn.Linear(in_features=1000, out_features=3))
        self.model.append(nn.Softmax(dim = 1))
        [self.add_module(name = "final_layer%d"%(idx), module = layer) for idx, layer in enumerate(self.model)]
        

    def forward(self, whole_input):
        out = whole_input
        for layer in self.model:
            out = layer(out)
        
        return out

In [158]:
import time
history = []
s = 0
N = 10
t = time.time()
for k in range(N):
    X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size = 30)
    model = Model(61)
    class_weight = [len([j for j in y_train if j[i] == 1]) for i in range(3)]
    class_weight = [1/j for j in class_weight]
    opt = torch.optim.Adam(model.parameters(), lr=1e-4)
    history = []
    for i in range(2400):
        ix = np.random.randint(0, len(X_train), 40) 
        x_batch = torch.tensor(X_train[ix], dtype=torch.float32)
        y_batch = torch.tensor(y_train[ix], dtype=torch.float32)

        y_predicted = model(x_batch)

        loss = F.binary_cross_entropy(y_predicted, y_batch, weight = torch.tensor(class_weight, dtype=torch.float32))

        loss.backward()

        opt.step()

        opt.zero_grad()

        history.append(loss.data.numpy())
    x = torch.tensor(X_test, dtype=torch.float32)
    
    y_pred = model(x).data.numpy()
    s += f1_score(y_pred.argmax(axis = 1), y_test.argmax(axis=1), average = 'macro')
    print(f1_score(y_pred.argmax(axis = 1), y_test.argmax(axis=1), average = 'macro'))
    if k%3 == 0:
        print(k, (time.time() - t)/(k+1), s/(k+1))
s / N

0.6888888888888889
0 46.77133083343506 0.6888888888888889
0.5655223329641933
0.4205128205128205
0.3470899470899471
3 49.510306000709534 0.5055034973639625
0.37306201550387597
0.367965367965368
0.3074074074074074
6 46.25251112665449 0.43863554004750016
0.3965744400527009
0.5833333333333333
0.6703703703703704
9 44.963056325912476 0.4720726924088905


0.4720726924088905